In [1]:
import pandas as pd

In [2]:
filepath = "C:/Users/28chr/Downloads/International_Dataset.xlsx"
data = pd.read_excel(filepath)

In [7]:
academic_challenge = {
    "Never" : 20,
    "Rarely" : 35,
    "Sometimes" : 50,
    "Often" : 65,
    "Very often" : 80
}
language_barriers = {
    "Strongly disagree" : 7,
    "Disagree" : 14,
    "Neutral" : 21,
    "Agree" : 28,
    "Strongly agree" : 35,
}

In [8]:
test_good_thresh = 0
test_good_compare = 0
test_tot = 0
for i in range(len(data["Timestamp"])):
    academic_score = 0
    personal_score = 0
    for word in str(data["Causes of Anxiety"][i]).split(", "):
        word = word.lower()
        if word == "academic pressure":
            academic_score = academic_challenge[data["Academics Challenging"][i]]
        elif word == "other academic":
            academic_score += 20
        elif word == "language barriers":
            personal_score += language_barriers[data["Language Barriers"][i]]
        # very few academic factors, the rest are personal
        else:
            personal_score += 20
    for word in str(data["Internal Stressors"][i]).split(", "):
        word = word.lower()
        if word == "other academic":
            academic_score += 20
        elif word == "pressure of publications":
            academic_score += 20
        else:
            personal_score += 20
    for word in str(data["External Stressors"][i]).split(", "):
        word = word.lower()
        if word == "other academic":
            academic_score += 20
        elif word == "pressure of publications":
            academic_score += 20
        else:
            personal_score += 20
    academic_difference_anxiety = 5 * data["Teaching Difference Anxiety"][i]
    if data["Impact of Difference"][i] == "Yes, created anxiety":
        academic_difference_anxiety *= 2
    academic_score += academic_difference_anxiety
    if academic_score >= 100 and data["Anxiety Cause"][i] == "Academic":
        test_good_thresh += 1
    elif personal_score >= 100 and data["Anxiety Cause"][i] == "Personal":
        test_good_thresh += 1
    if academic_score > personal_score and data["Anxiety Cause"][i] == "Academic":
        test_good_compare += 1
    elif personal_score > academic_score and data["Anxiety Cause"][i] == "Personal":
        test_good_compare += 1
    if data["Anxiety Cause"][i] != "Unsure":
        test_tot += 1
    print ("Academic:", academic_score)
    print ("Personal:", personal_score)
    print ("Designation:", data["Anxiety Cause"][i])

Academic: 130
Personal: 100
Designation: Academic
Academic: 5
Personal: 68
Designation: Personal
Academic: 65
Personal: 80
Designation: Unsure
Academic: 15
Personal: 88
Designation: Personal
Academic: 10
Personal: 60
Designation: Personal
Academic: 140
Personal: 188
Designation: Unsure
Academic: 80
Personal: 100
Designation: Unsure
Academic: 5
Personal: 60
Designation: Personal
Academic: 135
Personal: 154
Designation: Academic
Academic: 90
Personal: 20
Designation: Academic
Academic: 95
Personal: 40
Designation: Academic
Academic: 55
Personal: 100
Designation: Personal
Academic: 90
Personal: 108
Designation: Unsure
Academic: 130
Personal: 180
Designation: Unsure
Academic: 30
Personal: 75
Designation: Unsure
Academic: 120
Personal: 88
Designation: Unsure
Academic: 10
Personal: 60
Designation: Personal
Academic: 80
Personal: 128
Designation: Unsure
Academic: 10
Personal: 60
Designation: Personal
Academic: 5
Personal: 60
Designation: Personal
Academic: 145
Personal: 140
Designation: Unsur

In [9]:
print("Threshold successes:", test_good_thresh)
print("Comparison successes:", test_good_compare)
print("Test set size:", test_tot)

Threshold successes: 7
Comparison successes: 18
Test set size: 19


# Linear Regression with Domain Knowledge

In [10]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [12]:
# dictionaries for interpreting data
academics_challenging_res = {
    "Never" : 0,
    "Rarely" : 1,
    "Sometimes" : 2,
    "Often" : 3,
    "Very often" : 4
}

language_barriers_res = {
    "Strongly disagree" : 0,
    "Disagree" : 1,
    "Neutral" : 2,
    "Agree" : 3,
    "Strongly agree" : 4
}

teaching_difference_res = {
    "Very similar" : 0,
    "Somewhat similar" : 1, # No such responses
    "Neutral" : 2,
    "Somewhat different" : 3,
    "Extremely different" : 4
}

difference_impact_res = {
    "No emotional impact" : 0,
    "Yes, created excitement" : 0,
    "Yes, created anxiety" : 1
}

# Training data: All rows that could be classified as either personal or academic
training_set_personal = [1, 3, 4, 7, 11, 16, 18, 19, 28, 29, 33, 34, 36, 37]
training_set_academic = [0, 8, 9, 10, 22]

In [20]:
import random
# create three models with different training/test sets
regs = []
test_sets = []
for i in range(3):
    # indices of training data
    test_set_personal = random.sample(range(len(training_set_personal)), 3)
    # can only afford to have 1 test sample for academic due to few samples
    test_set_academic = random.randrange(len(training_set_academic))
    test_sets.append((test_set_personal, test_set_academic))
    training_data = []
    result_data = []
    for j in range(len(training_set_personal)):
        if not j in test_set_personal:
            this_training_data = []
            ind = training_set_personal[j]
            this_training_data.append(academics_challenging_res[data["Academics Challenging"][ind]])
            this_training_data.append(language_barriers_res[data["Language Barriers"][ind]])
            this_training_data.append(teaching_difference_res[data["Teaching Difference"][ind]])
            this_training_data.append(difference_impact_res[data["Impact of Difference"][ind]])
            this_training_data.append(data["Teaching Difference Anxiety"][ind])
            # Reduced to most common stressors or causes of anxiety based on domain knowledge
            # This should reduce overfit caused by too many factors
            academic_pressure = 0
            homesickness = 0
            language_barriers = 0
            cultural_differences = 0
            financial_issues = 0
            other_academic = 0
            other_personal = 0
            for word in str(data["Causes of Anxiety"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["Internal Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["External Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            this_training_data.append(academic_pressure)
            this_training_data.append(homesickness)
            this_training_data.append(language_barriers)
            this_training_data.append(cultural_differences)
            this_training_data.append(financial_issues)
            this_training_data.append(other_academic)
            this_training_data.append(other_personal)
            training_data.append(this_training_data)
            result_data.append(0)
    for j in range(len(training_set_academic)):
        if j != test_set_academic:
            this_training_data = []
            ind = training_set_academic[j]
            this_training_data.append(academics_challenging_res[data["Academics Challenging"][ind]])
            this_training_data.append(language_barriers_res[data["Language Barriers"][ind]])
            this_training_data.append(teaching_difference_res[data["Teaching Difference"][ind]])
            this_training_data.append(difference_impact_res[data["Impact of Difference"][ind]])
            this_training_data.append(data["Teaching Difference Anxiety"][ind])
            # Reduced to most common stressors or causes of anxiety based on domain knowledge
            # This should reduce overfit caused by too many factors
            academic_pressure = 0
            homesickness = 0
            language_barriers = 0
            cultural_differences = 0
            financial_issues = 0
            other_academic = 0
            other_personal = 0
            for word in str(data["Causes of Anxiety"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["Internal Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["External Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != "n/a":
                    other_personal += 1
            this_training_data.append(academic_pressure)
            this_training_data.append(homesickness)
            this_training_data.append(language_barriers)
            this_training_data.append(cultural_differences)
            this_training_data.append(financial_issues)
            this_training_data.append(other_academic)
            this_training_data.append(other_personal)
            training_data.append(this_training_data)
            result_data.append(1)
    training_data = np.array(training_data)
    result_data = np.array(result_data)
    reg = LinearRegression().fit(training_data, result_data)
    regs.append(reg)

In [24]:
results = [[], [], []]
for i in range(3):
    for ind in range(len(data["Timestamp"])):
        this_training_data = []
        this_training_data.append(academics_challenging_res[data["Academics Challenging"][ind]])
        this_training_data.append(language_barriers_res[data["Language Barriers"][ind]])
        this_training_data.append(teaching_difference_res[data["Teaching Difference"][ind]])
        this_training_data.append(difference_impact_res[data["Impact of Difference"][ind]])
        this_training_data.append(data["Teaching Difference Anxiety"][ind])
        # Reduced to most common stressors or causes of anxiety based on domain knowledge
        # This should reduce overfit caused by too many factors
        academic_pressure = 0
        homesickness = 0
        language_barriers = 0
        cultural_differences = 0
        financial_issues = 0
        other_academic = 0
        other_personal = 0
        for word in str(data["Causes of Anxiety"][ind]).split(", "):
            word = word.lower()
            if word == "academic pressure":
                academic_pressure += 1
            elif word == "homesickness":
                homesickness += 1
            elif word == "language barriers":
                language_barriers += 1
            elif word == "cultural differences":
                cultural_differences += 1
            elif word == financial_issues:
                financial_issues += 1
            elif word == "other academic":
                other_academic += 1
            # seperate in the dataset but does not merit its own category here
            elif word == "pressure of publications":
                other_academic += 1
            # most stressors and causes of anxiety are personal
            elif word != 'n/a':
                other_personal += 1
        for word in str(data["Internal Stressors"][ind]).split(", "):
            word = word.lower()
            if word == "academic pressure":
                academic_pressure += 1
            elif word == "homesickness":
                homesickness += 1
            elif word == "language barriers":
                language_barriers += 1
            elif word == "cultural differences":
                cultural_differences += 1
            elif word == financial_issues:
                financial_issues += 1
            elif word == "other academic":
                other_academic += 1
            # seperate in the dataset but does not merit its own category here
            elif word == "pressure of publications":
                other_academic += 1
            # most stressors and causes of anxiety are personal
            elif word != 'n/a':
                other_personal += 1
        for word in str(data["External Stressors"][ind]).split(", "):
            word = word.lower()
            if word == "academic pressure":
                academic_pressure += 1
            elif word == "homesickness":
                homesickness += 1
            elif word == "language barriers":
                language_barriers += 1
            elif word == "cultural differences":
                cultural_differences += 1
            elif word == financial_issues:
                financial_issues += 1
            elif word == "other academic":
                other_academic += 1
            # seperate in the dataset but does not merit its own category here
            elif word == "pressure of publications":
                other_academic += 1
            # most stressors and causes of anxiety are personal
            elif word != 'n/a':
                other_personal += 1
        this_training_data.append(academic_pressure)
        this_training_data.append(homesickness)
        this_training_data.append(language_barriers)
        this_training_data.append(cultural_differences)
        this_training_data.append(financial_issues)
        this_training_data.append(other_academic)
        this_training_data.append(other_personal)
        training = (ind in training_set_academic or ind in training_set_personal) and ind != training_set_personal[test_sets[i][0][0]] and ind != training_set_personal[test_sets[i][0][1]] and ind != training_set_personal[test_sets[i][0][2]] and ind != training_set_academic[test_sets[i][1]]
        results[i].append((regs[i].predict([this_training_data]), training, data["Anxiety Cause"][ind]))

In [25]:
for i in range(len(data["Timestamp"])):
    print("Model 1:")
    print(results[0][i])
    print("Model 2:")
    print(results[1][i])
    print("Model 3:")
    print(results[2][i])
    print()

Model 1:
(array([0.7389737]), False, 'Academic')
Model 2:
(array([1.04271583]), True, 'Academic')
Model 3:
(array([0.43645884]), False, 'Academic')

Model 1:
(array([0.05033025]), True, 'Personal')
Model 2:
(array([0.00199825]), True, 'Personal')
Model 3:
(array([-0.00757596]), True, 'Personal')

Model 1:
(array([0.34619339]), False, 'Unsure')
Model 2:
(array([-0.62351419]), False, 'Unsure')
Model 3:
(array([0.15045952]), False, 'Unsure')

Model 1:
(array([-0.32715136]), False, 'Personal')
Model 2:
(array([0.03590015]), True, 'Personal')
Model 3:
(array([-0.00555857]), True, 'Personal')

Model 1:
(array([-0.08874618]), True, 'Personal')
Model 2:
(array([-0.04190842]), True, 'Personal')
Model 3:
(array([-0.29765781]), False, 'Personal')

Model 1:
(array([0.82717384]), False, 'Unsure')
Model 2:
(array([2.60861597]), False, 'Unsure')
Model 3:
(array([0.68449136]), False, 'Unsure')

Model 1:
(array([0.97726571]), False, 'Unsure')
Model 2:
(array([1.53803776]), False, 'Unsure')
Model 3:
(ar

In [26]:
# Compute total inconsistency across the models
incon_linear = 0
for i in range(len(data["Timestamp"])):
    res1 = results[0][i][0]
    res2 = results[1][i][0]
    res3 = results[2][i][0]
    incon_linear += abs(res1 - res2) + abs(res2-res3) + abs(res1 - res3)
print(incon_linear)

[77.71052615]


In [ ]:
# Results: Linear regression has some discrepency but is relatively reliable. Updates to process helped reduce overfit.

# SVM

In [28]:
from sklearn import svm
# create three models with different training/test sets
svms = []
svm_test_sets = []
for i in range(3):
    # indices of training data
    test_set_personal = random.sample(range(len(training_set_personal)), 3)
    # can only afford to have 1 test sample for academic due to few samples
    test_set_academic = random.randrange(len(training_set_academic))
    svm_test_sets.append((test_set_personal, test_set_academic))
    training_data = []
    result_data = []
    for j in range(len(training_set_personal)):
        if not j in test_set_personal:
            this_training_data = []
            ind = training_set_personal[j]
            this_training_data.append(academics_challenging_res[data["Academics Challenging"][ind]])
            this_training_data.append(language_barriers_res[data["Language Barriers"][ind]])
            this_training_data.append(teaching_difference_res[data["Teaching Difference"][ind]])
            this_training_data.append(difference_impact_res[data["Impact of Difference"][ind]])
            this_training_data.append(data["Teaching Difference Anxiety"][ind])
            # Reduced to most common stressors or causes of anxiety based on domain knowledge
            # This should reduce overfit caused by too many factors
            academic_pressure = 0
            homesickness = 0
            language_barriers = 0
            cultural_differences = 0
            financial_issues = 0
            other_academic = 0
            other_personal = 0
            for word in str(data["Causes of Anxiety"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["Internal Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["External Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            this_training_data.append(academic_pressure)
            this_training_data.append(homesickness)
            this_training_data.append(language_barriers)
            this_training_data.append(cultural_differences)
            this_training_data.append(financial_issues)
            this_training_data.append(other_academic)
            this_training_data.append(other_personal)
            training_data.append(this_training_data)
            result_data.append(0)
    for j in range(len(training_set_academic)):
        if j != test_set_academic:
            this_training_data = []
            ind = training_set_academic[j]
            this_training_data.append(academics_challenging_res[data["Academics Challenging"][ind]])
            this_training_data.append(language_barriers_res[data["Language Barriers"][ind]])
            this_training_data.append(teaching_difference_res[data["Teaching Difference"][ind]])
            this_training_data.append(difference_impact_res[data["Impact of Difference"][ind]])
            this_training_data.append(data["Teaching Difference Anxiety"][ind])
            # Reduced to most common stressors or causes of anxiety based on domain knowledge
            # This should reduce overfit caused by too many factors
            academic_pressure = 0
            homesickness = 0
            language_barriers = 0
            cultural_differences = 0
            financial_issues = 0
            other_academic = 0
            other_personal = 0
            for word in str(data["Causes of Anxiety"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["Internal Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["External Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != "n/a":
                    other_personal += 1
            this_training_data.append(academic_pressure)
            this_training_data.append(homesickness)
            this_training_data.append(language_barriers)
            this_training_data.append(cultural_differences)
            this_training_data.append(financial_issues)
            this_training_data.append(other_academic)
            this_training_data.append(other_personal)
            training_data.append(this_training_data)
            result_data.append(1)
    training_data = np.array(training_data)
    result_data = np.array(result_data)
    reg = svm.SVC().fit(training_data, result_data)
    svms.append(reg)

In [29]:
svm_results = [[], [], []]
for i in range(3):
    for ind in range(len(data["Timestamp"])):
        this_training_data = []
        this_training_data.append(academics_challenging_res[data["Academics Challenging"][ind]])
        this_training_data.append(language_barriers_res[data["Language Barriers"][ind]])
        this_training_data.append(teaching_difference_res[data["Teaching Difference"][ind]])
        this_training_data.append(difference_impact_res[data["Impact of Difference"][ind]])
        this_training_data.append(data["Teaching Difference Anxiety"][ind])
        # Reduced to most common stressors or causes of anxiety based on domain knowledge
        # This should reduce overfit caused by too many factors
        academic_pressure = 0
        homesickness = 0
        language_barriers = 0
        cultural_differences = 0
        financial_issues = 0
        other_academic = 0
        other_personal = 0
        for word in str(data["Causes of Anxiety"][ind]).split(", "):
            word = word.lower()
            if word == "academic pressure":
                academic_pressure += 1
            elif word == "homesickness":
                homesickness += 1
            elif word == "language barriers":
                language_barriers += 1
            elif word == "cultural differences":
                cultural_differences += 1
            elif word == financial_issues:
                financial_issues += 1
            elif word == "other academic":
                other_academic += 1
            # seperate in the dataset but does not merit its own category here
            elif word == "pressure of publications":
                other_academic += 1
            # most stressors and causes of anxiety are personal
            elif word != 'n/a':
                other_personal += 1
        for word in str(data["Internal Stressors"][ind]).split(", "):
            word = word.lower()
            if word == "academic pressure":
                academic_pressure += 1
            elif word == "homesickness":
                homesickness += 1
            elif word == "language barriers":
                language_barriers += 1
            elif word == "cultural differences":
                cultural_differences += 1
            elif word == financial_issues:
                financial_issues += 1
            elif word == "other academic":
                other_academic += 1
            # seperate in the dataset but does not merit its own category here
            elif word == "pressure of publications":
                other_academic += 1
            # most stressors and causes of anxiety are personal
            elif word != 'n/a':
                other_personal += 1
        for word in str(data["External Stressors"][ind]).split(", "):
            word = word.lower()
            if word == "academic pressure":
                academic_pressure += 1
            elif word == "homesickness":
                homesickness += 1
            elif word == "language barriers":
                language_barriers += 1
            elif word == "cultural differences":
                cultural_differences += 1
            elif word == financial_issues:
                financial_issues += 1
            elif word == "other academic":
                other_academic += 1
            # seperate in the dataset but does not merit its own category here
            elif word == "pressure of publications":
                other_academic += 1
            # most stressors and causes of anxiety are personal
            elif word != 'n/a':
                other_personal += 1
        this_training_data.append(academic_pressure)
        this_training_data.append(homesickness)
        this_training_data.append(language_barriers)
        this_training_data.append(cultural_differences)
        this_training_data.append(financial_issues)
        this_training_data.append(other_academic)
        this_training_data.append(other_personal)
        training = (ind in training_set_academic or ind in training_set_personal) and ind != training_set_personal[svm_test_sets[i][0][0]] and ind != training_set_personal[svm_test_sets[i][0][1]] and ind != training_set_personal[svm_test_sets[i][0][2]] and ind != training_set_academic[svm_test_sets[i][1]]
        svm_results[i].append((svms[i].predict([this_training_data]), training, data["Anxiety Cause"][ind]))

In [30]:
for i in range(len(data["Timestamp"])):
    print("Model 1:")
    print(svm_results[0][i])
    print("Model 2:")
    print(svm_results[1][i])
    print("Model 3:")
    print(svm_results[2][i])
    print()

Model 1:
(array([1]), True, 'Academic')
Model 2:
(array([1]), True, 'Academic')
Model 3:
(array([0]), True, 'Academic')

Model 1:
(array([0]), True, 'Personal')
Model 2:
(array([0]), True, 'Personal')
Model 3:
(array([0]), False, 'Personal')

Model 1:
(array([0]), False, 'Unsure')
Model 2:
(array([0]), False, 'Unsure')
Model 3:
(array([0]), False, 'Unsure')

Model 1:
(array([0]), True, 'Personal')
Model 2:
(array([0]), False, 'Personal')
Model 3:
(array([0]), True, 'Personal')

Model 1:
(array([0]), True, 'Personal')
Model 2:
(array([0]), True, 'Personal')
Model 3:
(array([0]), True, 'Personal')

Model 1:
(array([0]), False, 'Unsure')
Model 2:
(array([0]), False, 'Unsure')
Model 3:
(array([0]), False, 'Unsure')

Model 1:
(array([0]), False, 'Unsure')
Model 2:
(array([0]), False, 'Unsure')
Model 3:
(array([0]), False, 'Unsure')

Model 1:
(array([0]), True, 'Personal')
Model 2:
(array([0]), True, 'Personal')
Model 3:
(array([0]), True, 'Personal')

Model 1:
(array([1]), True, 'Academic')

In [ ]:
# Results: SVM classifies everything not in the training set as personal. Completely inviable.

# Random Forest

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
# create three models with different training/test sets
forests = []
rf_test_sets = []
for i in range(3):
    # indices of training data
    test_set_personal = random.sample(range(len(training_set_personal)), 3)
    # can only afford to have 1 test sample for academic due to few samples
    test_set_academic = random.randrange(len(training_set_academic))
    rf_test_sets.append((test_set_personal, test_set_academic))
    training_data = []
    result_data = []
    for j in range(len(training_set_personal)):
        if not j in test_set_personal:
            this_training_data = []
            ind = training_set_personal[j]
            this_training_data.append(academics_challenging_res[data["Academics Challenging"][ind]])
            this_training_data.append(language_barriers_res[data["Language Barriers"][ind]])
            this_training_data.append(teaching_difference_res[data["Teaching Difference"][ind]])
            this_training_data.append(difference_impact_res[data["Impact of Difference"][ind]])
            this_training_data.append(data["Teaching Difference Anxiety"][ind])
            # Reduced to most common stressors or causes of anxiety based on domain knowledge
            # This should reduce overfit caused by too many factors
            academic_pressure = 0
            homesickness = 0
            language_barriers = 0
            cultural_differences = 0
            financial_issues = 0
            other_academic = 0
            other_personal = 0
            for word in str(data["Causes of Anxiety"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["Internal Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["External Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            this_training_data.append(academic_pressure)
            this_training_data.append(homesickness)
            this_training_data.append(language_barriers)
            this_training_data.append(cultural_differences)
            this_training_data.append(financial_issues)
            this_training_data.append(other_academic)
            this_training_data.append(other_personal)
            training_data.append(this_training_data)
            result_data.append(0)
    for j in range(len(training_set_academic)):
        if j != test_set_academic:
            this_training_data = []
            ind = training_set_academic[j]
            this_training_data.append(academics_challenging_res[data["Academics Challenging"][ind]])
            this_training_data.append(language_barriers_res[data["Language Barriers"][ind]])
            this_training_data.append(teaching_difference_res[data["Teaching Difference"][ind]])
            this_training_data.append(difference_impact_res[data["Impact of Difference"][ind]])
            this_training_data.append(data["Teaching Difference Anxiety"][ind])
            # Reduced to most common stressors or causes of anxiety based on domain knowledge
            # This should reduce overfit caused by too many factors
            academic_pressure = 0
            homesickness = 0
            language_barriers = 0
            cultural_differences = 0
            financial_issues = 0
            other_academic = 0
            other_personal = 0
            for word in str(data["Causes of Anxiety"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["Internal Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != 'n/a':
                    other_personal += 1
            for word in str(data["External Stressors"][ind]).split(", "):
                word = word.lower()
                if word == "academic pressure":
                    academic_pressure += 1
                elif word == "homesickness":
                    homesickness += 1
                elif word == "language barriers":
                    language_barriers += 1
                elif word == "cultural differences":
                    cultural_differences += 1
                elif word == financial_issues:
                    financial_issues += 1
                elif word == "other academic":
                    other_academic += 1
                # seperate in the dataset but does not merit its own category here
                elif word == "pressure of publications":
                    other_academic += 1
                # most stressors and causes of anxiety are personal
                elif word != "n/a":
                    other_personal += 1
            this_training_data.append(academic_pressure)
            this_training_data.append(homesickness)
            this_training_data.append(language_barriers)
            this_training_data.append(cultural_differences)
            this_training_data.append(financial_issues)
            this_training_data.append(other_academic)
            this_training_data.append(other_personal)
            training_data.append(this_training_data)
            result_data.append(1)
    training_data = np.array(training_data)
    result_data = np.array(result_data)
    reg = RandomForestClassifier().fit(training_data, result_data)
    forests.append(reg)

In [33]:
rf_results = [[], [], []]
for i in range(3):
    for ind in range(len(data["Timestamp"])):
        this_training_data = []
        this_training_data.append(academics_challenging_res[data["Academics Challenging"][ind]])
        this_training_data.append(language_barriers_res[data["Language Barriers"][ind]])
        this_training_data.append(teaching_difference_res[data["Teaching Difference"][ind]])
        this_training_data.append(difference_impact_res[data["Impact of Difference"][ind]])
        this_training_data.append(data["Teaching Difference Anxiety"][ind])
        # Reduced to most common stressors or causes of anxiety based on domain knowledge
        # This should reduce overfit caused by too many factors
        academic_pressure = 0
        homesickness = 0
        language_barriers = 0
        cultural_differences = 0
        financial_issues = 0
        other_academic = 0
        other_personal = 0
        for word in str(data["Causes of Anxiety"][ind]).split(", "):
            word = word.lower()
            if word == "academic pressure":
                academic_pressure += 1
            elif word == "homesickness":
                homesickness += 1
            elif word == "language barriers":
                language_barriers += 1
            elif word == "cultural differences":
                cultural_differences += 1
            elif word == financial_issues:
                financial_issues += 1
            elif word == "other academic":
                other_academic += 1
            # seperate in the dataset but does not merit its own category here
            elif word == "pressure of publications":
                other_academic += 1
            # most stressors and causes of anxiety are personal
            elif word != 'n/a':
                other_personal += 1
        for word in str(data["Internal Stressors"][ind]).split(", "):
            word = word.lower()
            if word == "academic pressure":
                academic_pressure += 1
            elif word == "homesickness":
                homesickness += 1
            elif word == "language barriers":
                language_barriers += 1
            elif word == "cultural differences":
                cultural_differences += 1
            elif word == financial_issues:
                financial_issues += 1
            elif word == "other academic":
                other_academic += 1
            # seperate in the dataset but does not merit its own category here
            elif word == "pressure of publications":
                other_academic += 1
            # most stressors and causes of anxiety are personal
            elif word != 'n/a':
                other_personal += 1
        for word in str(data["External Stressors"][ind]).split(", "):
            word = word.lower()
            if word == "academic pressure":
                academic_pressure += 1
            elif word == "homesickness":
                homesickness += 1
            elif word == "language barriers":
                language_barriers += 1
            elif word == "cultural differences":
                cultural_differences += 1
            elif word == financial_issues:
                financial_issues += 1
            elif word == "other academic":
                other_academic += 1
            # seperate in the dataset but does not merit its own category here
            elif word == "pressure of publications":
                other_academic += 1
            # most stressors and causes of anxiety are personal
            elif word != 'n/a':
                other_personal += 1
        this_training_data.append(academic_pressure)
        this_training_data.append(homesickness)
        this_training_data.append(language_barriers)
        this_training_data.append(cultural_differences)
        this_training_data.append(financial_issues)
        this_training_data.append(other_academic)
        this_training_data.append(other_personal)
        training = (ind in training_set_academic or ind in training_set_personal) and ind != training_set_personal[rf_test_sets[i][0][0]] and ind != training_set_personal[rf_test_sets[i][0][1]] and ind != training_set_personal[rf_test_sets[i][0][2]] and ind != training_set_academic[rf_test_sets[i][1]]
        rf_results[i].append((forests[i].predict([this_training_data]), training, data["Anxiety Cause"][ind]))

In [34]:
for i in range(len(data["Timestamp"])):
    print("Model 1:")
    print(rf_results[0][i])
    print("Model 2:")
    print(rf_results[1][i])
    print("Model 3:")
    print(rf_results[2][i])
    print()

Model 1:
(array([1]), True, 'Academic')
Model 2:
(array([1]), True, 'Academic')
Model 3:
(array([0]), False, 'Academic')

Model 1:
(array([0]), True, 'Personal')
Model 2:
(array([0]), True, 'Personal')
Model 3:
(array([0]), True, 'Personal')

Model 1:
(array([0]), False, 'Unsure')
Model 2:
(array([0]), False, 'Unsure')
Model 3:
(array([0]), False, 'Unsure')

Model 1:
(array([0]), True, 'Personal')
Model 2:
(array([0]), True, 'Personal')
Model 3:
(array([0]), True, 'Personal')

Model 1:
(array([0]), True, 'Personal')
Model 2:
(array([0]), True, 'Personal')
Model 3:
(array([0]), True, 'Personal')

Model 1:
(array([1]), False, 'Unsure')
Model 2:
(array([1]), False, 'Unsure')
Model 3:
(array([1]), False, 'Unsure')

Model 1:
(array([0]), False, 'Unsure')
Model 2:
(array([1]), False, 'Unsure')
Model 3:
(array([0]), False, 'Unsure')

Model 1:
(array([0]), True, 'Personal')
Model 2:
(array([0]), False, 'Personal')
Model 3:
(array([0]), True, 'Personal')

Model 1:
(array([1]), True, 'Academic')

In [ ]:
# Results: More consistent than linear regression, but may suffer from overfit and misclassified all three academic test data